In [59]:
import gzip


def unzip(input, output):

    input_file = input
    output_file = output

    with gzip.open(input_file, "rb") as f_in:
        with open(output_file, "wb") as f_out:

            f_out.write(f_in.read())

    print("file unzipped and save at", output_file)

unzip("/data/LSY/z_preparing_and_parts/commontrait_Parkinson/GSE72776_datBetaNormalized.csv.gz",
      "/data/LSY/z_preparing_and_parts/commontrait_Parkinson/GSE72776_series_matrix.txt")

file unzipped and save at /data/LSY/z_preparing_and_parts/commontrait_Parkinson/GSE72776_series_matrix.txt


In [13]:

import pandas as pd



# read matrix
with open(
    "/data/LSY/z_preparing_and_parts/commontrait_Erythematosus/GSE110607_series_matrix.txt", "r"
) as file:
    lines = file.readlines()

# find metadata
first_empty_line_index = lines.index("\n")
table_begin_index = next(
(
        i
        for i, line in enumerate(lines)
        if line.startswith("!series_matrix_table_begin")
    ),
    None,
)
end_index = table_begin_index if table_begin_index is not None else len(lines)
content_before_table = lines[first_empty_line_index:end_index]


# read metadata
metadata_lines = [
    line.strip()[1:].split("\t")
    for line in content_before_table
    if line.startswith("!")
]
metadata_df = pd.DataFrame(metadata_lines).T


metadata_df.columns = metadata_df.iloc[0]
metadata_df = metadata_df.drop(0, axis=0)

metadata_df.dropna(inplace=True)


In [41]:
# mask = metadata_df.iloc[:, 7] == '"Peripheral blood"'

# metadata_df = metadata_df[mask]



In [9]:
print(metadata_df.iloc[:,9].unique())

['"cell type: Monocytes"' '"cell type: B-cells"'
 '"cell type: CD4+ T-cells"' '"cell type: Granulocytes"']


In [14]:

# find status
metadata_df["status"] = metadata_df.iloc[:, 9].apply(
    lambda x: 1 if "erythematosus" in x else 0 if "Co-twin" in x else None
)
metadata_df = metadata_df.set_index("Sample_geo_accession")

In [15]:
metadata_df["group"] = metadata_df.iloc[:, 9].apply(
    lambda x: 1 if "Monocytes" in x else None
)

In [16]:

metadata_df["status"] = metadata_df["status"].apply(pd.to_numeric, errors="coerce")
metadata_df["group"] = metadata_df["group"].apply(pd.to_numeric, errors="coerce")

In [17]:
print(metadata_df)

0                    Sample_title            Sample_status  \
Sample_geo_accession                                         
"GSM3004023"           "1a_CD14M"  "Public on Mar 19 2018"   
"GSM3004024"           "1b_CD14M"  "Public on Mar 19 2018"   
"GSM3004025"           "3a_CD14M"  "Public on Mar 19 2018"   
"GSM3004026"           "3b_CD14M"  "Public on Mar 19 2018"   
"GSM3004027"           "5a_CD14M"  "Public on Mar 19 2018"   
...                           ...                      ...   
"GSM3004122"             "13b_GR"  "Public on Mar 19 2018"   
"GSM3004123"             "14a_GR"  "Public on Mar 19 2018"   
"GSM3004124"             "14b_GR"  "Public on Mar 19 2018"   
"GSM3004125"             "15a_GR"  "Public on Mar 19 2018"   
"GSM3004126"             "15b_GR"  "Public on Mar 19 2018"   

0                    Sample_submission_date Sample_last_update_date  \
Sample_geo_accession                                                  
"GSM3004023"                  "Feb 14 2018"        

In [18]:


# read data
data_lines = [line.strip().split() for line in lines if not line.startswith("!")]
data_df = pd.DataFrame(data_lines).T

data_df.columns = data_df.iloc[0]
data_df = data_df.drop(0, axis=0)

data_df = data_df.set_index('"ID_REF"')
data_df = data_df.iloc[:, 1:]
data_df.columns = [col.strip('"') for col in data_df.columns]

data_df = data_df.apply(pd.to_numeric, errors="coerce")





In [21]:
print(data_df)

              cg00000029  cg00000108  cg00000109  cg00000165  cg00000236  \
"ID_REF"                                                                   
"GSM3004023"    0.428322    0.940220    0.809152    0.214586    0.701453   
"GSM3004024"    0.470388    0.931020    0.727571    0.293964    0.665480   
"GSM3004025"    0.479421    0.930388    0.737337    0.214660    0.705344   
"GSM3004026"    0.458408    0.957971    0.790909    0.165569    0.673133   
"GSM3004027"    0.489322    0.904693    0.735866    0.225062    0.709032   
"GSM3004028"    0.422559    0.899729    0.720280    0.206897    0.703008   
"GSM3004029"    0.530494    0.923539    0.793340    0.241171    0.738110   
"GSM3004030"    0.587625    0.928329    0.753653    0.256524    0.583493   
"GSM3004031"    0.348551    0.891241    0.721386    0.213039    0.620606   
"GSM3004032"    0.441915    0.894031    0.725711    0.256043    0.635612   
"GSM3004033"    0.433364    0.922471    0.668790    0.228979    0.542691   
"GSM3004034"

In [9]:

def m_to_beta(M):
    return 1 / (1 + 2 ** (-M))


In [10]:
# data_df = data_df.apply(m_to_beta)

In [20]:

# merge metadata and data
data_df["status"] = metadata_df["status"]
data_df["group"] = metadata_df["group"]
# drop other disease sample
data_df.dropna(subset=["group"], inplace=True)
data_df.dropna(subset=["status"], inplace=True)
data_df.drop("group", axis=1, inplace=True)


In [22]:

# save data
data_df.to_csv(
    "/data/LSY/z_preparing_and_parts/commontrait_Erythematosus/data_valid.txt",
    index=True,
    header=True,
    sep="\t",
)
